## Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os

In [2]:
url = 'https://books.toscrape.com/catalogue/page-1.html'
print(url)
req = requests.get(url)
soup = BeautifulSoup(req.content , 'html.parser')

https://books.toscrape.com/catalogue/page-1.html


## Scrapping Function

In [3]:
def Scrapping(url):
    
    for pgn in range(1 , 51):

        url = f'https://books.toscrape.com/catalogue/page-{pgn}.html'
        req = requests.get(url)
        soup = BeautifulSoup(req.content , 'html.parser')
        
        
        sec = soup.find('section')
        rows = sec.find('ol').find_all('li' , attrs = {'class' : 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})

        titles , rates , prices = [] , [] , []

        for row in rows:

            rate = row.find('p',attrs={'class':'star-rating'}).attrs['class'][-1]
            rates.append(rate)

            title = row.find('h3').find('a').attrs['title']
            titles.append(title)

            price = float((row.find('div',attrs={'class':'product_price'}).find('p',attrs={'class':'price_color'}).text)[1:])
            prices.append(price)

#             print(rate)
#             print(title)
#             print(price)

#             print('*'*30)


        pgn_num = ''

        if 'index' in url:
            try:
                pgn_num = int(soup.find('ul' , attrs = {'class' : 'pager'}).find('li' , attrs = {'class' : 'current'}).text.strip().split(' ')[-1])
            except:
                pass

        return titles , rates , prices , pgn_num


## Storing_Data Function

In [4]:
def store_data(cat_name , titles , rates , prices):
    
    m = 'w' if os.path.exists('Books_Information.csv') == False else 'a'
    
    with open('Books_Information.csv' , mode = m , newline='' , encoding='utf-8') as f :
        
        writer = csv.writer(f)
        
        if m == 'w':
            
            writer.writerow(['Category','Title','Price','Rate'])
            
        for tit , rat , price in zip(titles , rates , prices):
            
            writer.writerow([cat_name,tit,price,rat])

##  scrapping Categories

In [5]:
cat_outer_div = soup.find('div',attrs={'class':'side_categories'})
rows_cat = cat_outer_div.find('ul',attrs={'class':'nav nav-list'}).find_all('li')[1:]
categories = {rowcat.text.strip() :'https://books.toscrape.com/catalogue/' + rowcat.find('a')['href']  for rowcat in rows_cat}

In [6]:
for cat_name , url in categories.items():
    
    print(url)
    print('*'*40)
    
    titles , rates , prices , pgn_num = Scrapping(url)
    
    store_data(cat_name , titles , rates , prices)
    
    
    if type(pgn_num) == int: 
        
        for pgn in range(2 , pgn_num + 1 ):
            
            new_url = url.replace('index' , f'page-{pgn}')
            
            print(new_url)
            print('*'*40)
            
            titles , rates , prices , _ = Scrapping(new_url)
            store_data(cat_name , titles , rates , prices)

https://books.toscrape.com/catalogue/category/books/travel_2/index.html
****************************************
https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
****************************************
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
****************************************
https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
****************************************
https://books.toscrape.com/catalogue/category/books/classics_6/index.html
****************************************
https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html
****************************************
https://books.toscrape.com/catalogue/category/books/romance_8/index.html
****************************************
https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html
****************************************
https://books.toscrape.com/catalogue/category/books/fiction_

In [7]:
data = pd.read_csv('Books_Information.csv')
data

,Category,Title,Price,Rate
0,Travel,A Light in the Attic,51.77,Three
1,Travel,Tipping the Velvet,53.74,One
2,Travel,Soumission,50.10,One
3,Travel,Sharp Objects,47.82,Four
4,Travel,Sapiens: A Brief History of Humankind,54.23,Five
...,...,...,...,...
1995,Crime,Our Band Could Be Your Life: Scenes from the A...,57.25,Three
1996,Crime,Olio,23.88,One
1997,Crime,Mesaerion: The Best Science Fiction Stories 18...,37.59,One
1998,Crime,Libertarianism for Beginners,51.33,Two
